## LIMPEZA DO CSV

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

df = pd.read_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_raw.csv')
df.head()

,ID,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,...,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,1,5.0,51,19.0,11/05/06,30/03/06,0.12,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,2,3.5,50,14.0,25/05/05,19/05/05,0.02,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,3,0.0,0,NaN,12/06/01,08/05/05,-4.05,SULA,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,4,8.1,65,19.0,15/10/09,21/04/09,0.50,NaN,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,7,40.0,151,18.0,14/01/08,14/08/95,12.89,SAME,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


### Retirando as colunas que não influênciam no diagnóstico

In [97]:
df1 = df.drop(columns=['ID', 'Convenio',])
df1.head()
df1['SEXO'].value_counts()

SEXO
M                6065
F                4477
Indeterminado    1417
Masculino         584
Feminino          247
masculino          79
Name: count, dtype: int64

### Concertando as incoerências na escrita

In [99]:
dicionario_sexo = {
    'M': 'M',
    'F': 'F',
    'masculino': 'M',
    'Masculino': 'M',     
    'Feminino': 'F',    
    'Indeterminado': 'I',
    'I': 'I'
}

# Aplicando o mapeamento
df1['SEXO'] = df1['SEXO'].map(dicionario_sexo)
df1['SEXO'].value_counts()

SEXO
M    6728
F    4724
I    1417
Name: count, dtype: int64

### Tratamento da Coluna IDADE

como temos muitos dados errados, melhor calcular a idade usando a data de nascimento e a data do atendimento

In [100]:
# Converter colunas de data para datetime
df1['DN'] = pd.to_datetime(df1['DN'], format='%d/%m/%y', errors='coerce')
df1['Atendimento'] = pd.to_datetime(df1['Atendimento'], format='%d/%m/%y', errors='coerce')

# Calcular idade em anos: (Data Atendimento - Data Nascimento) / 365.25 dias
df1['IDADE'] = (df1['Atendimento'] - df1['DN']).dt.days / 365.25

# Transformar valores negativos em nulo
df1.loc[df1['IDADE'] < 0, 'IDADE'] = np.nan

# Verificar resultado
print("Primeiras linhas com datas convertidas e idade calculada:")
print(df1[['DN', 'Atendimento', 'IDADE']].head(10))
print("\n\nEstatísticas da IDADE calculada:")
print(df1['IDADE'].describe())
print("\n\nValores nulos em IDADE:")
print(df1['IDADE'].isna().sum())
df1['IDADE'].value_counts()

Primeiras linhas com datas convertidas e idade calculada:
          DN Atendimento      IDADE
0 2006-03-30  2006-05-11   0.114990
1 2005-05-19  2005-05-25   0.016427
2 2005-05-08  2001-06-12        NaN
3 2009-04-21  2009-10-15   0.484600
4 1995-08-14  2008-01-14  12.418891
5 1999-12-28  2005-09-01   5.678303
6 2006-03-20  2006-06-19   0.249144
7 2007-06-20  2007-09-06   0.213552
8 2001-11-29  2007-12-03   6.009582
9 2000-06-18  2003-06-20   3.003422


Estatísticas da IDADE calculada:
count    11072.000000
mean         5.140292
std          4.790232
min          0.000000
25%          0.988364
50%          3.889117
75%          8.298426
max         71.770021
Name: IDADE, dtype: float64


Valores nulos em IDADE:
1801


IDADE
0.002738     154
0.005476     113
0.008214      34
0.038330      31
0.024641      28
            ... 
9.127995       1
5.752225       1
11.997262      1
15.359343      1
23.460643      1
Name: count, Length: 4434, dtype: int64

### Tratando a coluna da variável alvo

aqui, todas as linhas que forem nulas, precisamos deletar, pois essa é a variável alvo que queremos prever

In [101]:
df1['NORMAL X ANORMAL'].isna().sum()

np.int64(1168)

In [102]:
df1.dropna(subset=['NORMAL X ANORMAL'], inplace=True)
df1['NORMAL X ANORMAL'].value_counts()

NORMAL X ANORMAL
Normal     6744
Anormal    4959
anormal       1
Normais       1
Name: count, dtype: int64

### Tratando coluna PULSOS

In [103]:
lista_indices = [
    12562, 12578, 12581, 12590, 12607, 12616, 12634, 12636, 
    12662, 12667, 12688, 12708, 12738, 12748, 12758, 12763, 
    12767, 12782, 12803, 12833, 12839, 12855
]

df1.drop(index=lista_indices, inplace=True)
df1['PULSOS'].value_counts()

PULSOS
Normais                11509
Amplos                    52
Femorais diminuidos       39
Outro                     36
Diminuídos                18
NORMAIS                    2
AMPLOS                     1
Name: count, dtype: int64

deletando linhas que tem tudo praticamente nulo

In [104]:
df1['PULSOS'] = df1['PULSOS'].str.strip().str.capitalize()
df1['PULSOS'] = df1['PULSOS'].fillna('Normais')
print(df1['PULSOS'].value_counts())

PULSOS
Normais                11537
Amplos                    53
Femorais diminuidos       39
Outro                     36
Diminuídos                18
Name: count, dtype: int64


### Tratando colunas Motivo

In [105]:
def limpar_motivo(valor):
    if pd.isna(valor):
        return np.nan
    
    texto = str(valor)

    texto_limpo = re.sub(r'^\d+\s*-\s*', '', texto)

    return texto_limpo.strip().lower()

df1['MOTIVO1'] = df1['MOTIVO1'].apply(limpar_motivo)
df1['MOTIVO2'] = df1['MOTIVO2'].apply(limpar_motivo)

df1['MOTIVO'] = df1['MOTIVO2'].fillna(df1['MOTIVO1'])

df1.drop(columns=['MOTIVO1', 'MOTIVO2'], inplace=True)
df1.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO
0,5.0,51,19.0,2006-05-11,2006-03-30,0.114990,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016427,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,check-up
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484600,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,parecer cardiológico
4,40.0,151,18.0,2008-01-14,1995-08-14,12.418891,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,parecer cardiológico


### Tratando colunas HDA

In [106]:
def unir_sintomas(row):
    sintomas = set()

    if pd.notna(row['HDA 1']):
        sintoma1 = str(row['HDA 1']).strip().lower()
        if sintoma1 not in ['', 'nan']:
            sintomas.add(sintoma1)

    if pd.notna(row['HDA2']):
        sintoma2 = str(row['HDA2']).strip().lower()
        if sintoma2 not in ['', 'nan']:
            sintomas.add(sintoma2)

    return ",".join(sintomas) 

df1['HDA'] = df1.apply(unir_sintomas, axis=1)

df1.drop(columns=['HDA 1', 'HDA2'], inplace=True)

print("Coluna unida:")
print(df1['HDA'].head())

Coluna unida:
0        palpitacao
1          dispneia
2     assintomático
3     assintomático
4    dor precordial
Name: HDA, dtype: object


In [107]:
df1['HDA'].value_counts()

HDA
assintomático                     6532
                                  3118
dor precordial                     406
dispneia                           400
palpitacao                         306
cianose                            197
desmaio/tontura                    124
outro                              123
ganho de peso                       79
cianose,dispneia                    75
dor precordial,palpitacao           71
dor precordial,dispneia             62
cianose,desmaio/tontura             27
palpitacao,dispneia                 27
outro,dispneia                      16
dor precordial,desmaio/tontura      15
dispneia,palpitacao                 14
ganho de peso,dispneia              14
desmaio/tontura,palpitacao          11
cianose,palpitacao                   8
outro,desmaio/tontura                8
dor precordial,cianose               7
desmaio/tontura,dispneia             7
dor precordial,ganho de peso         6
dor precordial,outro                 5
dispneia,desmaio/tont

### Tratando a coluna PPA

In [108]:
def atualizar_ppa(df, drop_pressure_columns=True):
    """Atualiza a coluna PPA usando PA SISTOLICA, PA DIASTOLICA e IDADE já existentes no DataFrame.
    Se PPA estiver nulo ou com 'Não Calculado', recalcula. Caso contrário mantém.
    Optionally remove the raw pressure columns.
    """
    bp_ref = {
        1:  {'M': {'S90': 100, 'S95': 104, 'D90': 53, 'D95': 58}, 'F': {'S90': 100, 'S95': 105, 'D90': 55, 'D95': 59}},
        2:  {'M': {'S90': 104, 'S95': 108, 'D90': 58, 'D95': 62}, 'F': {'S90': 104, 'S95': 108, 'D90': 60, 'D95': 65}},
        3:  {'M': {'S90': 107, 'S95': 110, 'D90': 62, 'D95': 66}, 'F': {'S90': 104, 'S95': 108, 'D90': 64, 'D95': 68}},
        4:  {'M': {'S90': 109, 'S95': 112, 'D90': 66, 'D95': 70}, 'F': {'S90': 106, 'S95': 110, 'D90': 67, 'D95': 71}},
        5:  {'M': {'S90': 110, 'S95': 114, 'D90': 69, 'D95': 73}, 'F': {'S90': 107, 'S95': 111, 'D90': 69, 'D95': 73}},
        6:  {'M': {'S90': 111, 'S95': 115, 'D90': 71, 'D95': 75}, 'F': {'S90': 109, 'S95': 113, 'D90': 70, 'D95': 74}},
        7:  {'M': {'S90': 113, 'S95': 117, 'D90': 73, 'D95': 77}, 'F': {'S90': 111, 'S95': 115, 'D90': 72, 'D95': 76}},
        8:  {'M': {'S90': 114, 'S95': 118, 'D90': 74, 'D95': 79}, 'F': {'S90': 113, 'S95': 116, 'D90': 73, 'D95': 77}},
        9:  {'M': {'S90': 115, 'S95': 119, 'D90': 76, 'D95': 80}, 'F': {'S90': 114, 'S95': 118, 'D90': 74, 'D95': 78}},
        10: {'M': {'S90': 117, 'S95': 121, 'D90': 76, 'D95': 81}, 'F': {'S90': 116, 'S95': 120, 'D90': 75, 'D95': 79}},
        11: {'M': {'S90': 119, 'S95': 123, 'D90': 76, 'D95': 81}, 'F': {'S90': 118, 'S95': 122, 'D90': 76, 'D95': 80}},
        12: {'M': {'S90': 121, 'S95': 125, 'D90': 77, 'D95': 82}, 'F': {'S90': 120, 'S95': 124, 'D90': 77, 'D95': 81}},
        13: {'M': {'S90': 124, 'S95': 128, 'D90': 77, 'D95': 82}, 'F': {'S90': 122, 'S95': 126, 'D90': 78, 'D95': 82}},
        14: {'M': {'S90': 126, 'S95': 130, 'D90': 78, 'D95': 83}, 'F': {'S90': 124, 'S95': 127, 'D90': 79, 'D95': 83}},
        15: {'M': {'S90': 129, 'S95': 133, 'D90': 79, 'D95': 84}, 'F': {'S90': 125, 'S95': 129, 'D90': 80, 'D95': 84}},
        16: {'M': {'S90': 131, 'S95': 135, 'D90': 81, 'D95': 86}, 'F': {'S90': 126, 'S95': 130, 'D90': 81, 'D95': 85}},
        17: {'M': {'S90': 134, 'S95': 138, 'D90': 83, 'D95': 87}, 'F': {'S90': 126, 'S95': 130, 'D90': 81, 'D95': 85}}
    }

    def calcular_ppa(row):
        # Se faltar algum dado essencial, mantém valor atual
        if pd.isna(row.get('PA SISTOLICA')) or pd.isna(row.get('PA DIASTOLICA')) or pd.isna(row.get('IDADE')):
            return row.get('PPA')

        try:
            idade = int(row['IDADE'])
        except (ValueError, TypeError):
            return row.get('PPA')

        if idade < 1:
            return 'Requer Tabela Infantil'
        if idade > 17:
            idade = 17

        sexo = str(row.get('SEXO', '')).strip().upper()
        s_key = 'M' if sexo in ['M', 'MASCULINO', 'BOY'] else 'F'

        limites = bp_ref.get(idade, bp_ref[17])[s_key]
        pas = row['PA SISTOLICA']
        pad = row['PA DIASTOLICA']

        if pas >= limites['S95'] or pad >= limites['D95']:
            return 'Hipertensão'
        elif pas >= limites['S90'] or pad >= limites['D90'] or pas >= 120 or pad >= 80:
            return 'Elevada'
        else:
            return 'Normal'

    print('Calculando PPA...')
    df['PPA_Calculada'] = df.apply(calcular_ppa, axis=1)

    mask_update = (df['PPA'].isna()) | (df['PPA'] == 'Não Calculado')
    df.loc[mask_update, 'PPA'] = df.loc[mask_update, 'PPA_Calculada']

    df.drop(columns=['PPA_Calculada'], inplace=True)

    n_nans = df['PPA'].isna().sum() + (df['PPA'] == 'Não Calculado').sum()
    print(f'Processo concluído. NaNs restantes em PPA: {n_nans}')

    if drop_pressure_columns:
        df.drop(columns=['PA SISTOLICA', 'PA DIASTOLICA'], inplace=True, errors='ignore')
        print("Colunas ['PA SISTOLICA', 'PA DIASTOLICA'] removidas.")

    return df

# Aplicar no df1 já carregado e limpo
# Se quiser manter as colunas de pressão, passe drop_pressure_columns=False
# Ex: df1 = atualizar_ppa(df1, drop_pressure_columns=False)
df1 = atualizar_ppa(df1)

# Para salvar tudo depois (descomente a linha abaixo):
# df1.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_limpo.csv', index=False)


Calculando PPA...
Processo concluído. NaNs restantes em PPA: 6320
Colunas ['PA SISTOLICA', 'PA DIASTOLICA'] removidas.


In [109]:
df1

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
0,5.0,51,19.0,2006-05-11,2006-03-30,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático
4,40.0,151,18.0,2008-01-14,1995-08-14,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12868,9.5,76,16.0,2006-02-03,2004-12-06,1.160849,Normais,Não Calculado,Normal,Normal,ausente,92,M,cirurgia,assintomático
12869,12.0,75,21.0,2009-09-25,2008-12-15,0.777550,Normais,Não Calculado,Normal,Normal,ausente,90,M,cirurgia,assintomático
12870,65.0,175,21.0,2007-08-03,1984-02-16,23.460643,Normais,Elevada,Normal,Normal,ausente,76,F,cardiopatia congenica,dor precordial
12871,27.0,134,15.0,2004-10-04,2027-12-03,NaN,Normais,Não Calculado,Normal,Normal,Sistólico,NaN,F,dor precordial,


### Tratando a coluna B2

In [110]:
df1['B2'].value_counts()

B2
Normal           11077
Hiperfonética      295
Desdob fixo        149
Única               77
Outro               76
Name: count, dtype: int64

In [111]:
df1['B2'] = df1['B2'].fillna('Normal')

In [112]:
df1.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
0,5.0,51,19.0,2006-05-11,2006-03-30,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático
4,40.0,151,18.0,2008-01-14,1995-08-14,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial


### Tratando coluna SOPRO

In [113]:
df1['SOPRO'].isna().sum()

np.int64(1)

In [114]:
df1['SOPRO'].value_counts()

SOPRO
ausente                   7260
Sistólico                 3640
sistólico                  723
contínuo                    28
Contínuo                    19
diastólico                   9
Sistolico e diastólico       3
Name: count, dtype: int64

In [115]:
df1[df1['SOPRO'].isna()]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
1951,0.0,0,NaN,2009-04-08,2003-08-21,5.631759,Outro,Não Calculado,Normal,Normal,NaN,NaN,F,cirurgia,


In [116]:
df1.dropna(subset=['SOPRO'], inplace=True)

In [117]:
df1.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
0,5.0,51,19.0,2006-05-11,2006-03-30,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático
4,40.0,151,18.0,2008-01-14,1995-08-14,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial


### Retirando linhas onde os dados faltantes são majoritários

In [118]:
(df1.isna().mean() * 100).sort_values(ascending=False)

IMC                 33.145009
IDADE                8.594419
DN                   7.079267
FC                   5.957884
Atendimento          4.417052
Peso                 2.653655
MOTIVO               2.268447
PPA                  0.102722
SEXO                 0.034241
Altura               0.000000
PULSOS               0.000000
NORMAL X ANORMAL     0.000000
B2                   0.000000
SOPRO                0.000000
HDA                  0.000000
dtype: float64

In [119]:
# 1. Calcula a porcentagem de nulos para cada linha
percentual_nulos_linha = df1.isna().mean(axis=1)

# 2. Filtra apenas as linhas com mais de 50% (0.5) de nulos
linhas_vazias = df1[percentual_nulos_linha > 0.3]

# Mostra quantas linhas foram encontradas
print(f"Foram encontradas {len(linhas_vazias)} linhas com mais de 50% de dados faltantes.")

# Visualiza essas linhas (mostrando apenas as primeiras 5 para não poluir a tela)
linhas_vazias.head()

Foram encontradas 154 linhas com mais de 50% de dados faltantes.


,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
56,NaN,116,NaN,NaT,NaT,NaN,Normais,Normal,Normal,Normal,ausente,88,M,dor precordial,assintomático
380,0.0,0,NaN,NaT,NaT,NaN,Normais,#VALUE!,Normal,Normal,ausente,96,I,NaN,
536,0.0,0,NaN,NaT,NaT,NaN,Normais,#VALUE!,Anormal,Hiperfonética,Sistólico,NaN,F,NaN,
560,0.0,0,NaN,NaT,NaT,NaN,Normais,#VALUE!,Normal,Normal,ausente,96,I,NaN,
695,0.0,0,NaN,NaT,NaT,NaN,Normais,#VALUE!,Normal,Normal,ausente,96,I,NaN,


In [120]:
# 1. Identifica os índices das linhas com mais de 30% (0.5) de nulos
linhas_para_remover = df1[df1.isna().mean(axis=1) > 0.3].index

# 2. Remove as linhas usando .drop()
df1.drop(linhas_para_remover, inplace=True)

# 3. Feedback para você saber quantas foram apagadas
print(f"Foram removidas {len(linhas_para_remover)} linhas.")

Foram removidas 154 linhas.


In [121]:
df1.drop(columns=['DN', 'Atendimento'], inplace=True)

In [123]:
df1.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
0,5.0,51,19.0,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia
2,0.0,0,NaN,NaN,Normais,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático
3,8.1,65,19.0,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático
4,40.0,151,18.0,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial


In [124]:
df1.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_limpo.csv', index=False)

### Tratando as colunas numéricas

In [125]:
cols_biologicas = ['Peso', 'Altura', 'IDADE']

for col in cols_biologicas:
    if col in df1.columns:
        qtd_erros = df1[df1[col] <= 0].shape[0]
        if qtd_erros > 0:
            print(f"Coluna '{col}': {qtd_erros} valores inválidos (<=0) transformados em NaN.")
            
            df1.loc[df1[col] <= 0, col] = np.nan


mask_imc = df1['Peso'].notnull() & df1['Altura'].notnull() & df1['IMC'].isnull()
df1.loc[mask_imc, 'IMC'] = df1.loc[mask_imc, 'Peso'] / ((df1.loc[mask_imc, 'Altura']/100) ** 2)

for col in cols_biologicas + ['IMC']:
    if col in df1.columns:
        mediana = df1[col].median()
        df1[col] = df1[col].fillna(mediana)

print("\nVerificação Final de Nulos:")
print(df1[cols_biologicas].isnull().sum())


Coluna 'Peso': 1649 valores inválidos (<=0) transformados em NaN.
Coluna 'Altura': 3461 valores inválidos (<=0) transformados em NaN.
Coluna 'IDADE': 27 valores inválidos (<=0) transformados em NaN.

Verificação Final de Nulos:
Peso      0
Altura    0
IDADE     0
dtype: int64


In [ ]:
# Cálculo de IMC (Peso kg / Altura(m)^2) e estatísticas
required_cols = {'Peso','Altura','SEXO','IDADE'}
missing = required_cols - set(df1.columns)
if missing:
    raise ValueError(f'Colunas ausentes para cálculo IMC: {missing}')

# Máscara de linhas válidas
mask_valid = (
    df1['Peso'].notna() & df1['Altura'].notna() &
    (df1['Peso'] > 0) & (df1['Altura'] > 0)
)

# Altura em metros
altura_m = df1.loc[mask_valid, 'Altura'] / 100.0
imc_values = df1.loc[mask_valid, 'Peso'] / (altura_m ** 2)

# Cria/atualiza coluna IMC_CALCULADO
df1.loc[mask_valid, 'IMC_CALCULADO'] = imc_values

# Se existir IMC original, completa valores ausentes com calculados
if 'IMC' in df1.columns:
    df1['IMC'] = df1['IMC'].where(df1['IMC'].notna(), df1['IMC_CALCULADO'])
    imc_col_final = 'IMC'
else:
    imc_col_final = 'IMC_CALCULADO'

print('Estatísticas gerais IMC:')
print(df1[imc_col_final].describe())

print('\nEstatísticas por SEXO:')
print(df1.groupby('SEXO')[imc_col_final].describe())

# Categorização simplificada (adulto) – não substitui percentil pediátrico
def classificar_imc(imc):
    if pd.isna(imc):
        return np.nan
    if imc < 18.5:
        return 'Baixo'
    elif imc < 25:
        return 'Normal'
    elif imc < 30:
        return 'Sobrepeso'
    else:
        return 'Obesidade'

df1['IMC_CATEGORIA_SIMPLIFICADA'] = df1[imc_col_final].apply(classificar_imc)

print('\nDistribuição categoria IMC simplificada:')
print(df1['IMC_CATEGORIA_SIMPLIFICADA'].value_counts(dropna=True))

Estatísticas gerais IMC:
count    11528.000000
mean        17.295802
std         12.597427
min          0.000000
25%         16.000000
50%         17.000000
75%         18.000000
max        848.000000
Name: IMC, dtype: float64

Estatísticas por SEXO:
       count       mean        std  min   25%   50%   75%    max
SEXO                                                            
F     4511.0  17.367324  16.969428  0.0  16.0  17.0  17.0  848.0
I      530.0  15.520755   3.326661  0.0  13.0  16.0  17.0   30.0
M     6483.0  17.390406   8.981081  0.0  16.0  17.0  18.0  612.0

Distribuição categoria IMC simplificada:
IMC_CATEGORIA_SIMPLIFICADA
Baixo        9340
Normal       1869
Sobrepeso     254
Obesidade      65
Name: count, dtype: int64


np.int64(0)

In [130]:
qtd_both_zero = df1[(df1['Peso'] == 0) & (df1['Altura'] == 0)].shape[0]
qtd_peso_zero = df1[df1['Peso'] == 0].shape[0]
qtd_altura_zero = df1[df1['Altura'] == 0].shape[0]
print(f"Linhas com Peso=0 e Altura=0: {qtd_both_zero}")
print(f"Linhas com Peso=0 (independente da altura): {qtd_peso_zero}")
print(f"Linhas com Altura=0 (independente do peso): {qtd_altura_zero}")

Linhas com Peso=0 e Altura=0: 0
Linhas com Peso=0 (independente da altura): 0
Linhas com Altura=0 (independente do peso): 0


In [131]:
df1

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA,IMC_CALCULADO,IMC_CATEGORIA_SIMPLIFICADA
0,5.0,51.0,19.0,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao,19.223376,Normal
1,3.5,50.0,14.0,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia,14.000000,Baixo
2,16.0,99.0,17.0,3.819302,Normais,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático,16.324865,Baixo
3,8.1,65.0,19.0,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático,19.171598,Normal
4,40.0,151.0,18.0,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial,17.543090,Baixo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12868,9.5,76.0,16.0,1.160849,Normais,Não Calculado,Normal,Normal,ausente,92,M,cirurgia,assintomático,16.447368,Baixo
12869,12.0,75.0,21.0,0.777550,Normais,Não Calculado,Normal,Normal,ausente,90,M,cirurgia,assintomático,21.333333,Normal
12870,65.0,175.0,21.0,23.460643,Normais,Elevada,Normal,Normal,ausente,76,F,cardiopatia congenica,dor precordial,21.224490,Normal
12871,27.0,134.0,15.0,3.819302,Normais,Não Calculado,Normal,Normal,Sistólico,NaN,F,dor precordial,,15.036757,Baixo


In [136]:
# Remover coluna IMC original (se existir) e renomear IMC_CALCULADO -> IMC
if 'IMC' in df1.columns:
    df1.drop(columns=['IMC'], inplace=True)
    print('Coluna IMC original removida.')
else:
    print('Coluna IMC original não encontrada (nada a remover).')

if 'IMC_CALCULADO' in df1.columns:
    df1.rename(columns={'IMC_CALCULADO': 'IMC'}, inplace=True)
    print('Coluna IMC_CALCULADO renomeada para IMC.')
else:
    print('Coluna IMC_CALCULADO não encontrada para renomear.')

print('Colunas atuais:', sorted(df1.columns))

Coluna IMC original removida.
Coluna IMC_CALCULADO renomeada para IMC.
Colunas atuais: ['Altura', 'B2', 'FC', 'HDA', 'IDADE', 'IMC', 'IMC_CATEGORIA_SIMPLIFICADA', 'MOTIVO', 'NORMAL X ANORMAL', 'PPA', 'PULSOS', 'Peso', 'SEXO', 'SOPRO']


### Tratando FC

In [137]:
def clean_fc(val):
    if pd.isna(val):
        return np.nan
    
    val_str = str(val).strip()
    
    if '-' in val_str:
        try:
            parts = val_str.split('-')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan
    if ' a ' in val_str:
        try:
            parts = val_str.split(' a ')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan

    try:
        num = float(val_str)
    except ValueError:
        return np.nan

    if num > 300: 
        if num < 3000: # Provável erro de x10 (ex: 1120 -> 112)
             num = num / 10
        elif num < 30000: # Provável erro de x100 (ex: 9288 -> 92.88)
             num = num / 100
        else:
             return np.nan # Dado irrecuperável

    if num < 30: 
        return np.nan

    return round(num)

df1['FC'] = df1['FC'].apply(clean_fc)

In [138]:
df1['Age_Int'] = df1['IDADE'].apply(lambda x: int(x) if pd.notnull(x) else x)

df1['FC'] = df1['FC'].fillna(df1.groupby('Age_Int')['FC'].transform('median'))

df1['FC'] = df1['FC'].fillna(df1['FC'].median())

df1.drop(columns=['Age_Int'], inplace=True)

In [139]:
df1['FC'].describe()

count    11528.000000
mean        95.270082
std         17.902940
min         43.000000
25%         80.000000
50%         92.000000
75%        100.000000
max        300.000000
Name: FC, dtype: float64

In [140]:
df1.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_limpo.csv', index=False)

### Tratando valores ausentes gerais

In [146]:
df1.isna().sum()

Peso                           0
Altura                         0
IDADE                          0
PULSOS                         0
PPA                           12
NORMAL X ANORMAL               0
B2                             0
SOPRO                          0
FC                             0
SEXO                           0
MOTIVO                         0
HDA                            0
IMC                            0
IMC_CATEGORIA_SIMPLIFICADA     0
dtype: int64

In [142]:
df1['HDA'] = df1['HDA'].fillna('Não Informado')

In [143]:
def preencher_motivo(row):
    if pd.notna(row['MOTIVO']):
        return row['MOTIVO']
    
    hda = str(row['HDA']).lower()
    
    if 'dispneia' in hda:
        return 'dispnéia'  
    elif 'dor precordial' in hda:
        return 'dor precordial'
    elif 'palpitacao' in hda or 'palpitação' in hda:
        return 'palpitação/taquicardia/arritmia'
    elif 'cianose' in hda:
        return 'cianose'
    elif 'assintomático' in hda:
        return 'parecer cardiológico' 
    else:
        return 'Não Informado'

df1['MOTIVO'] = df1.apply(preencher_motivo, axis=1)

print("Valores ausentes restantes em MOTIVO:", df1['MOTIVO'].isnull().sum())
print(df1['MOTIVO'].value_counts())

Valores ausentes restantes em MOTIVO: 0
MOTIVO
cirurgia                           2844
sopro                              2283
parecer cardiológico               1744
cardiopatia congenica              1014
outro                               783
check-up                            654
atividade física                    585
palpitação/taquicardia/arritmia     399
dor precordial                      373
dispnéia                            229
has/dislipidemia/obesidade          225
cianose                             140
cardiopatia adquirida                90
suspeita de cardiopatia              59
Não Informado                        51
cardiopatia na familia               32
uso de cisaprida                      9
cansaço                               6
alterações de pulso/perfusão          5
cianose e dispnéia                    3
Name: count, dtype: int64


In [145]:
df1.dropna(subset=['SEXO'], inplace=True)

In [147]:
df1.dropna(subset=['PPA'], inplace=True)

In [149]:
df1.count()

Peso                          11512
Altura                        11512
IDADE                         11512
PULSOS                        11512
PPA                           11512
NORMAL X ANORMAL              11512
B2                            11512
SOPRO                         11512
FC                            11512
SEXO                          11512
MOTIVO                        11512
HDA                           11512
IMC                           11512
IMC_CATEGORIA_SIMPLIFICADA    11512
dtype: int64

In [150]:
df1.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_limpo.csv', index=False)